## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,-38.0023,-57.5575,64.87,91,100,23.00,overcast clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,79.56,37,25,9.86,scattered clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,19.24,63,40,19.57,scattered clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,55.06,58,75,5.75,broken clouds
4,4,Chokurdakh,RU,70.6333,147.9167,-39.28,100,99,2.21,overcast clouds


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bredasdorp,ZA,-34.5322,20.0403,79.56,37,25,9.86,scattered clouds
6,6,East London,ZA,-33.0153,27.9116,75.67,80,4,23.53,clear sky
8,8,Gao,ML,16.6362,1.6370,81.37,8,0,10.16,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,78.78,71,91,16.08,overcast clouds
14,14,Puerto Padre,CU,21.1950,-76.6028,83.41,43,0,13.78,clear sky
16,16,Arraial Do Cabo,BR,-22.9661,-42.0278,87.66,58,0,18.41,clear sky
21,21,Mahebourg,MU,-20.4081,57.7000,79.25,82,40,12.66,scattered clouds
27,27,Birao,CF,10.2849,22.7882,76.01,12,58,7.90,broken clouds
28,28,Cape Town,ZA,-33.9258,18.4232,78.10,42,19,1.01,few clouds
29,29,Maceio,BR,-9.6658,-35.7353,89.04,51,20,16.11,few clouds


In [51]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bredasdorp,ZA,79.56,scattered clouds,-34.5322,20.0403,
6,East London,ZA,75.67,clear sky,-33.0153,27.9116,
8,Gao,ML,81.37,clear sky,16.6362,1.6370,
9,Rikitea,PF,78.78,overcast clouds,-23.1203,-134.9692,
14,Puerto Padre,CU,83.41,clear sky,21.1950,-76.6028,
16,Arraial Do Cabo,BR,87.66,clear sky,-22.9661,-42.0278,
21,Mahebourg,MU,79.25,scattered clouds,-20.4081,57.7000,
27,Birao,CF,76.01,broken clouds,10.2849,22.7882,
28,Cape Town,ZA,78.10,few clouds,-33.9258,18.4232,
29,Maceio,BR,89.04,few clouds,-9.6658,-35.7353,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [55]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df[hotel_df["Hotel Name"]!=""]

In [56]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bredasdorp,ZA,79.56,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
6,East London,ZA,75.67,clear sky,-33.0153,27.9116,Tu Casa
9,Rikitea,PF,78.78,overcast clouds,-23.1203,-134.9692,People ThankYou
14,Puerto Padre,CU,83.41,clear sky,21.1950,-76.6028,Casa Villa Velázquez
16,Arraial Do Cabo,BR,87.66,clear sky,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
663,Venice,US,72.00,mist,27.0998,-82.4543,A Beach Retreat On Casey Key
664,Waingapu,ID,79.47,scattered clouds,-9.6567,120.2641,Pondok Wisata Elim
673,Pacocha,PE,73.71,overcast clouds,-17.5833,-71.3333,terrenos en venta
675,Rantauprapat,ID,73.87,overcast clouds,2.1000,99.8333,Permata Land Hotel & Resort


In [57]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [60]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [61]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))